# RO param. covariance

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
import scipy.stats
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import cartopy.util
import copy

# Import custom modules
import src.utils
from src.XRO import XRO, xcorr

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## Load data

In [ ]:
## fits
save_fp = pathlib.Path(SAVE_FP, "fits_cesm", "T3_hw_bymember.nc")
# save_fp = pathlib.Path(SAVE_FP, "fits_cesm", "T3_hwhat_bymember.nc")
fits = xr.open_dataset(save_fp)

## Th data
Th = src.utils.load_cesm_indices()

## omit first year (bc of NaN in h,hw vars)
Th = Th.sel(time=slice("1851", None))

## standardize (for convenience)
Th /= Th.std()

## get windowed data (used to estimate change in parameters over time)
Th_rolling = src.utils.get_windowed(Th, window_size=480, stride=120)

## extract param values

In [ ]:
## specify model
MODEL = src.XRO.XRO(ncycle=12, ac_order=3, is_forward=True)

## extract parameters
params = src.utils.get_params(fits=fits, model=MODEL)

## get change from initial period
delta_params = params - params.isel(year=0)

## Plot stats

### Plotting funcs

In [ ]:
def scatter_params(ax, p0, p1, params, **scatter_kwargs):
    """scatter params on ax object"""

    ## helper func to select data
    sel = lambda n: params[n].mean("cycle")

    ## scatter data
    ax.scatter(sel(p0), sel(p1), **scatter_kwargs)

    ## title
    r = xr.corr(sel(p0), sel(p1)).values.item()
    ax.set_title(f"$r$({p0}, {p1})$=${r:.2f}")

    ## label
    ax.set_xlabel(p0)
    ax.set_ylabel(p1)

    return ax


def compare_scatter_params(axs, params, **scatter_kwargs):
    """comparison plot of parameters"""

    ## shared args
    kwargs = dict(params=params, **scatter_kwargs)

    ## scatter the parameters
    scatter_params(axs[0], "R", "F1", **kwargs)
    scatter_params(axs[1], "epsilon", "F2", **kwargs)
    scatter_params(axs[2], "F1", "F2", **kwargs)
    scatter_params(axs[3], "R", "epsilon", **kwargs)
    scatter_params(axs[4], "BJ_ac", "wyrtki", **kwargs)

    return

### make plots

In [ ]:
print(f"Early:")
fig, axs = plt.subplots(1, 5, figsize=(11, 2), layout="constrained")
compare_scatter_params(axs, params.isel(year=0), s=15)
plt.show()

print(f"\n\nLate:")
fig, axs = plt.subplots(1, 5, figsize=(11, 2), layout="constrained")
compare_scatter_params(axs, params.isel(year=-1), s=15)
plt.show()

print(f"\n\nChange:")
fig, axs = plt.subplots(1, 5, figsize=(11, 2), layout="constrained")
compare_scatter_params(axs, delta_params.isel(year=-1), s=15)
ax_kwargs = dict(ls="--", c="k", lw=0.8)
for ax in axs:
    ax.axhline(0, **ax_kwargs)
    ax.axvline(0, **ax_kwargs)

plt.show()